<a href="https://colab.research.google.com/github/sairam2000/Color-Detection/blob/master/colorizeimages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import tensorflow as tf

In [3]:
print(tf.__version__)

2.4.1


In [4]:
PATH = '/content/gdrive/MyDrive/dataset/'

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
trainGenerator = ImageDataGenerator(rescale=1/255.)

In [7]:
traindata = trainGenerator.flow_from_directory(PATH,target_size=(224,224),batch_size=100,class_mode=None)

Found 106 images belonging to 1 classes.


In [8]:
from tensorflow.keras.preprocessing.image import img_to_array

In [9]:
import os

In [10]:
images = os.listdir('/content/gdrive/MyDrive/dataset/ds')
print(len(images))

106


In [11]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread(PATH+images[100])
resized_img = cv2.resize(img,(244,244))
cv2_imshow(resized_img)


error: ignored

In [12]:
from tensorflow.keras.layers import Conv2D,MaxPool2D,UpSampling2D,Input
from tensorflow.keras.models import Model,Sequential

In [13]:
from tensorflow.keras.applications.vgg16 import VGG16

In [14]:
vgg = VGG16()

In [15]:
newmodel = Sequential()

In [16]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [17]:
for i,layer in enumerate(vgg.layers):
  if i< 19:
    newmodel.add(layer)

In [18]:
newmodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [19]:
for layer in newmodel.layers:
  layer.trainable = False

In [20]:
newmodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [21]:
print(len(traindata[0]))

6


In [22]:
from skimage.color import rgb2lab, lab2rgb, gray2rgb
from skimage.transform import resize
import numpy as np

In [23]:
X = []
Y = []
for i in traindata[0]:
  lab = rgb2lab(i)
  X.append(lab[:,:,0])
  Y.append(lab[:,:,1:]/128.)
X = np.array(X)
Y = np.array(Y)

print(X.shape)
print(Y.shape)


(100, 224, 224)
(100, 224, 224, 2)


In [24]:
X = X.reshape(X.shape+(1,))
print(X.shape)
print(Y.shape)

(100, 224, 224, 1)
(100, 224, 224, 2)


In [25]:
vggfeatures = []
for i,sample in enumerate(X):
  sample = gray2rgb(sample)
  sample = sample.reshape((1,224,224,3))
  prediction = newmodel.predict(sample)
  prediction = prediction.reshape(7,7,512)
  vggfeatures.append(prediction)

In [26]:
vggfeatures = np.array(vggfeatures)
print(vggfeatures.shape)

(100, 7, 7, 512)


In [27]:
model = Sequential()

model.add(Conv2D(256, (3,3), activation='relu', padding='same', input_shape=(7,7,512)))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 7, 7, 256)         1179904   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         295040    
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        73792     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 56, 56, 32)       

In [ ]:
model.compile(optimizer='Adam', loss='mse' , metrics=['accuracy'])
model.fit(vggfeatures, Y, verbose=1, epochs=1000, batch_size=128)

In [49]:
test = img_to_array(load_img('/content/1.jfif'))
test = resize(test, (224,224), anti_aliasing=True)
# imsave('/content/'+"2.jpg", test)
test*= 1.0/255
lab = rgb2lab(test)
l = lab[:,:,0]
L = gray2rgb(l)
imsave('/content/'+"20.jpg", L)
L = L.reshape((1,224,224,3))
#print(L.shape)
vggpred = newmodel.predict(L)
ab = model.predict(vggpred)
#print(ab.shape)
ab = ab*128
cur = np.zeros((224, 224, 3))
cur[:,:,0] = l
cur[:,:,1:] = ab
imsave('/content/'+"21.jpg", lab2rgb(cur))

In [32]:
from tensorflow.keras.preprocessing.image import img_to_array,load_img

In [36]:
from skimage.io import imsave